In [1]:
#Import libraries
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
#Memeriksa info file untuk melihat missing values dan data type
df_customer = pd.read_csv("F:/RAKAMIN/KALBE/final task/Customer.csv", delimiter=';')
df_product = pd.read_csv("F:/RAKAMIN/KALBE/final task/Product.csv", delimiter=';')
df_store = pd.read_csv("F:/RAKAMIN/KALBE/final task/Store.csv", delimiter=';')
df_transaction = pd.read_csv("F:/RAKAMIN/KALBE/final task/Transaction.csv", delimiter=';')


df_customer.info()
df_product.info()
df_store.info()
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CustomerID      447 non-null    int64 
 1   Age             447 non-null    int64 
 2   Gender          447 non-null    int64 
 3   Marital Status  444 non-null    object
 4   Income          447 non-null    object
dtypes: int64(3), object(2)
memory usage: 17.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ProductID     10 non-null     object
 1   Product Name  10 non-null     object
 2   Price         10 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 368.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      ------------

In [3]:
#Handling missing value
#dari info di atas, diketahui bahwa df_customer memilki 3 missing values di Marital Status
    # --> diisi dengan value berdasarkan case rata-rata usia dan marital status yang telah didapatkan di DBeaver sebelumnya.

#Memeriksa rows dengan missing value
null_marital_status = df_customer[df_customer['Marital Status'].isnull()]
print(null_marital_status)
    
# Define the function 
def fill_marital_status(row):
    if row['Marital Status'] not in ('Married', 'Single') and row['Age'] > 31:
        return 'Married'
    else:
        return 'Single'

# Apply the function to fill missing values
df_customer['Marital Status'] = df_customer.apply(fill_marital_status, axis=1)
df_customer.info()

     CustomerID  Age  Gender Marital Status Income
9            10   34       1            NaN      4
415         416   27       1            NaN   3,43
442         443   33       1            NaN   9,28
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CustomerID      447 non-null    int64 
 1   Age             447 non-null    int64 
 2   Gender          447 non-null    int64 
 3   Marital Status  447 non-null    object
 4   Income          447 non-null    object
dtypes: int64(3), object(2)
memory usage: 17.6+ KB


In [4]:
#Just to make sure the function has worked well
# List of customer IDs to check
customer_ids_to_check = [10, 416, 443]

# Filter dan print marital status yang baru
marital_status_check = df_customer[df_customer['CustomerID'].isin(customer_ids_to_check)][['CustomerID', 'Age','Marital Status']]
print(marital_status_check)

     CustomerID  Age Marital Status
9            10   34        Married
415         416   27         Single
442         443   33        Married


In [5]:
#Memeriksa Data Duplikat

#Kolom kunci penentuan duplicate row ada pada TransactionID sebagai nilai unik, jadi perlu dicek duplikatnya
df_transaction['TransactionID'].value_counts()

TR71313    3
TR42197    2
TR90402    2
TR74019    2
TR49649    2
          ..
TR29792    1
TR23081    1
TR1691     1
TR98388    1
TR85684    1
Name: TransactionID, Length: 4908, dtype: int64

In [6]:
#memeriksa isi data duplikat
df_transaction[df_transaction['TransactionID']=='TR71313']

,TransactionID,CustomerID,Date,ProductID,Price,Qty,TotalAmount,StoreID
1982,TR71313,117,21/05/2022,P1,8800,10,88000,8
3336,TR71313,401,30/08/2022,P3,7500,6,45000,11
3722,TR71313,370,26/09/2022,P3,7500,2,15000,3


In [7]:
#Hasil menunjukkan bahwa duplikasi hanya pada kolom TransactionID saja, lainnya berbeda.
    #Hal ini mungkin karena TransactionID digenerate dari store yang berbeda. 
    #Sehingga datanya masih valid dan dipertahankan 

In [8]:
#DATA INTEGRATION

#Data akan lebih mudah diidentifikasi untuk data cleansing dan transformation ketika sudah disatukan. 
#Oleh karena itu, dilakukan merge ke-empat data tsb lebih dahulu

#MERGE THE DATA
df_merged = pd.merge(df_transaction, df_customer, on=["CustomerID"])
df_merged = pd.merge(df_merged, df_product.drop(columns=['Price']), on=["ProductID"])
df_merged = pd.merge(df_merged, df_store, on=["StoreID"])

df_merged.info()
df_merged.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5020 entries, 0 to 5019
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TransactionID   5020 non-null   object
 1   CustomerID      5020 non-null   int64 
 2   Date            5020 non-null   object
 3   ProductID       5020 non-null   object
 4   Price           5020 non-null   int64 
 5   Qty             5020 non-null   int64 
 6   TotalAmount     5020 non-null   int64 
 7   StoreID         5020 non-null   int64 
 8   Age             5020 non-null   int64 
 9   Gender          5020 non-null   int64 
 10  Marital Status  5020 non-null   object
 11  Income          5020 non-null   object
 12  Product Name    5020 non-null   object
 13  StoreName       5020 non-null   object
 14  GroupStore      5020 non-null   object
 15  Type            5020 non-null   object
 16  Latitude        5020 non-null   object
 17  Longitude       5020 non-null   object
dtypes: int64

,TransactionID,CustomerID,Date,ProductID,Price,Qty,TotalAmount,StoreID,Age,Gender,Marital Status,Income,Product Name,StoreName,GroupStore,Type,Latitude,Longitude
0,TR11369,328,01/01/2022,P3,7500,4,30000,12,36,0,Single,"10,53",Crackers,Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554"
1,TR89318,183,17/07/2022,P3,7500,1,7500,12,27,1,Single,"0,18",Crackers,Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554"
2,TR9106,123,26/09/2022,P3,7500,4,30000,12,34,0,Single,"4,36",Crackers,Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554"
3,TR4331,335,08/01/2022,P3,7500,3,22500,12,29,1,Single,"4,74",Crackers,Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554"
4,TR6445,181,10/01/2022,P3,7500,4,30000,12,33,1,Single,"9,94",Crackers,Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554"


In [9]:
#DATA TRANSFORMATION

# Mengubah kolom "Date" ke tipe data datetime
df_merged['Date'] = pd.to_datetime(df_merged['Date'], format='%d/%m/%Y')

# Column customerid, gender, storeid dalam integer, kita ubah ke string/object karena tidak untuk dikalkulasi
columns_to_str = ['CustomerID', 'Gender', 'StoreID']
df_merged[columns_to_str] = df_merged[columns_to_str].astype(str)

# Column income, latitude, longitude dalam object, kita perlu ubah menjadi float dengan mengubah separator koma ke titik juga. 
df_merged['Income'] = df_merged['Income'].str.replace(',', '.').astype(float)
df_merged['Latitude'] = df_merged['Latitude'].str.replace(',', '.').astype(float)
df_merged['Longitude'] = df_merged['Longitude'].str.replace(',', '.').astype(float)


# Print data types untuk mengkonfirmasi transformasi
print(df_merged.dtypes)
print(df_merged.columns)

TransactionID             object
CustomerID                object
Date              datetime64[ns]
ProductID                 object
Price                      int64
Qty                        int64
TotalAmount                int64
StoreID                   object
Age                        int64
Gender                    object
Marital Status            object
Income                   float64
Product Name              object
StoreName                 object
GroupStore                object
Type                      object
Latitude                 float64
Longitude                float64
dtype: object
Index(['TransactionID', 'CustomerID', 'Date', 'ProductID', 'Price', 'Qty',
       'TotalAmount', 'StoreID', 'Age', 'Gender', 'Marital Status', 'Income',
       'Product Name', 'StoreName', 'GroupStore', 'Type', 'Latitude',
       'Longitude'],
      dtype='object')


In [10]:
#Memeriksa korelasi antar varibel
df_merged.corr()

,Price,Qty,TotalAmount,Age,Income,Latitude,Longitude
Price,1.000000,-0.353640,0.440632,0.014693,0.001196,-0.029008,0.018652
Qty,-0.353640,1.000000,0.621129,-0.027768,-0.028425,-0.004170,0.004807
TotalAmount,0.440632,0.621129,1.000000,-0.016900,-0.025350,-0.029938,0.025437
Age,0.014693,-0.027768,-0.016900,1.000000,0.486692,0.009266,0.015951
Income,0.001196,-0.028425,-0.025350,0.486692,1.000000,0.015518,0.004385
Latitude,-0.029008,-0.004170,-0.029938,0.009266,0.015518,1.000000,-0.395995
Longitude,0.018652,0.004807,0.025437,0.015951,0.004385,-0.395995,1.000000


In [11]:
#TotalAmount berkolerasi moderate positive dengan Qty. Tetap kita coba pakai untuk clustering. 

#Membuat df baru untuk clustering. Merge kolom TransactionID, Qty, dan TotalAmount berdasarkan CustomerID 
df_cluster=df_merged.groupby(['CustomerID']).agg({
    'TransactionID':'count',
    'Qty':'sum',
    'TotalAmount':'sum'
}).reset_index()

df_cluster.head()

,CustomerID,TransactionID,Qty,TotalAmount
0,1,17,60,623300
1,10,14,50,478000
2,100,8,35,272400
3,101,14,44,439600
4,102,15,57,423300


In [12]:
#Normalisasi data
#Drop kolom CustomerID karena tidak diperlukan dalam normalisasi
data_cluster=df_cluster.drop(columns=['CustomerID'])
data_cluster_normalize=preprocessing.normalize(data_cluster)
data_cluster_normalize

array([[2.72741856e-05, 9.62618317e-05, 9.99999995e-01],
       [2.92887028e-05, 1.04602510e-04, 9.99999994e-01],
       [2.93685754e-05, 1.28487517e-04, 9.99999991e-01],
       ...,
       [4.24886188e-05, 1.09256448e-04, 9.99999993e-01],
       [2.48275861e-05, 9.37931030e-05, 9.99999995e-01],
       [2.84450062e-05, 1.04298356e-04, 9.99999994e-01]])

In [13]:
#Performing KMeans Clustering
K = range(3, 8)
fits = []
score = []

for k in K:
    model = KMeans(n_clusters=k, random_state=0, n_init=10).fit(data_cluster_normalize)
    fits.append(model)
    score.append(silhouette_score(data_cluster_normalize, model.labels_, metric='euclidean'))

# Print silhouette scores untuk nilai k yang berbeda
for k, s in zip(K, score):
    print(f"Number of clusters (k): {k}, Silhouette Score: {s}")

Number of clusters (k): 3, Silhouette Score: 0.4665299778542393
Number of clusters (k): 4, Silhouette Score: 0.4562149955621574
Number of clusters (k): 5, Silhouette Score: 0.40401573583799977
Number of clusters (k): 6, Silhouette Score: 0.3672037759966429
Number of clusters (k): 7, Silhouette Score: 0.34995713268124023


In [14]:
#Memilih cluster k=3 dengan Silhouette Score tertinggi.
#Make sure the fits index
fits[0]

KMeans(n_clusters=3, random_state=0)

In [15]:
#menambakan kolom baru dengan nama 'cluster_label' ke 'df_cluster'
df_cluster['cluster_label']=fits[0].labels_

In [16]:
#Grouping cluster
df_cluster.groupby('cluster_label').agg({
    'CustomerID': 'count',
    'TransactionID': 'mean',
    'Qty':'mean',
    'TotalAmount': 'sum'
})


,CustomerID,TransactionID,Qty,TotalAmount
cluster_label,,,,
0,181,11.049724,41.558011,61632500
1,61,10.344262,39.229508,16387800
2,205,11.653659,40.882927,84022700


In [17]:
#Terdapat 3 cluster 0,1,2 